In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math 
import functools
import matplotlib.animation as animation
import matplotlib.text as matText

from matplotlib.collections import PatchCollection
from matplotlib.patches import Arc as patchArc

%matplotlib auto
#plt.rcParams["animation.html"] = "jshtml"
FRAMES_PER_NUMBER = 20
MAX_NUMBER_TO_CHECK = 5
plotCenter = (0,0)
artists_to_draw_manually = []


fig = plt.figure()
ax = fig.subplots()

numberDisplay = fig.text(.5, .9,'2', fontsize='large', animated=True)
artists_to_draw_manually.append(numberDisplay)

xSpine = ax.spines['bottom']
xSpine.set_position('zero')
#xSpine.set_bounds(0, None)
backGroundImage = None

class CircleCycleForOnePrime:
	primeNumber = 0
	center = (0,0)
#	center = (0.5,0.5)

	logScaling = False
	radius = 0
	currentIndicatorValue = 0
	RING_WIDTH = 2
	INDICATOR_WIDTH = RING_WIDTH * 2
	ANGLE_TO_INDICATE_ZERO = 270.0
	zeroWedge = None
	nonZeroWedge = None
	indicatorWedge = None
	modIndicatorZero=False
	unitSlice = None
	axesForCycle = None

	def __init__(self, center, primeNumber, axes, logScaling):
		self.center = center
		self.primeNumber = int( primeNumber)
		self.setLogScaling(logScaling)
		self.axesForCycle = axes
		self.constructPrimeModCirclePatches()
	
	def setLogScaling(self, useLogScaling, minimumRadius=0):
		self.logScaling = useLogScaling
		self.radius = 2* self.primeNumber
		if (self.logScaling):
			self.radius = math.log(self.primeNumber)
		if (self.radius < minimumRadius):
			self.radius = minimumRadius + 2
		

	
	def constructPrimeModCirclePatches(self):
		#divide a circle into primeNumber slices
		self.unitSlice = 360.0 / self.primeNumber
		zeroRegionBegin= 0  
		zeroRegionEnd = self.unitSlice

		self.zeroWedge=patchArc(self.center, self.radius, self.radius, theta1=zeroRegionBegin, theta2=zeroRegionEnd, linewidth=CircleCycleForOnePrime.RING_WIDTH, angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='red', animated=False)
		self.axesForCycle.add_patch(self.zeroWedge)

		self.nonZeroWedge=patchArc(self.center, self.radius, self.radius, theta1=zeroRegionEnd, theta2=zeroRegionBegin,linewidth=CircleCycleForOnePrime.RING_WIDTH,angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='green', animated=False)
		self.axesForCycle.add_patch(self.nonZeroWedge)
		
		self.indicatorWedge = patchArc(self.center, self.radius, self.radius, theta1= -self.unitSlice / 5, theta2= self.unitSlice/5, linewidth=CircleCycleForOnePrime.INDICATOR_WIDTH, angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='blue', animated=True)
		self.axesForCycle.add_patch(self.indicatorWedge)
		artists_to_draw_manually.append(self.indicatorWedge)
		self.setIndicator(0)


	def setIndicator(self, newIndicatorValue):
		#convert the remainder to a percent
		self.currentIndicatorValue = (newIndicatorValue % self.primeNumber)
		self.modIndicatorZero = (self.currentIndicatorValue == 0)
		translatedFromValueToAngle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2) + self.unitSlice*self.currentIndicatorValue
		self.indicatorWedge.set_angle(translatedFromValueToAngle)
		self.axesForCycle.draw_artist(self.indicatorWedge)


def drawArtistCollection(artistCollection):
	for oneArtist in artistCollection:
#		print("drawing artist " ,oneArtist)
		ax.draw_artist(oneArtist)


#The main drawing method
def updateClockForNumber(currentNumber, cycleContainer, artists_to_draw_manually):
	global backGroundImage
	# reset the background back in the canvas state, screen unchanged
	fig.canvas.restore_region(backGroundImage)
	#only checxk for a new cycle if this is an integer
	currentNumberIsIntegerAndNotDivisibleByAnyCycle = (currentNumber.is_integer() )
	setCountNumberDisplayed(currentNumber)
	for oneCycle in cycleContainer:
		oneCycle.setIndicator(currentNumber)
		#check if current number is divisible by cycle prime
		currentNumberIsIntegerAndNotDivisibleByAnyCycle = currentNumberIsIntegerAndNotDivisibleByAnyCycle and not oneCycle.modIndicatorZero
		
	#if current number isn't divisible by any cycle, it's a prime
	if (currentNumberIsIntegerAndNotDivisibleByAnyCycle):
		#got a new prime
		
		newCycle = CircleCycleForOnePrime(plotCenter, currentNumber, ax, False)
		cycles.append(newCycle)
		ax.autoscale_view()
		# flush any pending GUI events, re-painting the screen if needed
		fig.canvas.flush_events()
		backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)
    # re-render the artist, updating the canvas state, but not the screen
	drawArtistCollection(artists_to_draw_manually)
    # copy the image to the GUI state, but screen might not be changed yet
	fig.canvas.blit(fig.bbox)
	
def setCountNumberDisplayed(currentNumber):
	countingString = '{:2g}'.format(currentNumber)
	numberDisplay.set_text(countingString)
	ax.draw_artist(numberDisplay)

cycles = []
startingValue=2
plt.show(block=False)
print("about to make circle")
twoCycle = CircleCycleForOnePrime(plotCenter, startingValue, ax, False)
cycles.append(twoCycle)
ax.autoscale_view()
# flush any pending GUI events, re-painting the screen if needed
fig.canvas.flush_events()
backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)
print("done 2 cycle")

# draw the animated artist, this uses a cached renderer
drawArtistCollection(artists_to_draw_manually)
# show the result to the screen, this pushes the updated RGBA buffer from the
# renderer to the GUI framework so you can see it
fig.canvas.blit(fig.bbox)

#driving update loop
numberOfFrames = FRAMES_PER_NUMBER * MAX_NUMBER_TO_CHECK
for animationFrameIndex in range(numberOfFrames):
	currentNumber = startingValue + animationFrameIndex / FRAMES_PER_NUMBER
	updateClockForNumber(currentNumber, cycles, artists_to_draw_manually)
	# you can put a pause in if you want to slow things down
	plt.pause(.1)

Using matplotlib backend: qtagg
about to make circle
done 2 cycle


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math 
import functools
import matplotlib.animation as animation
import matplotlib.text as matText

from matplotlib.collections import PatchCollection
from matplotlib.patches import Arc as patchArc

%matplotlib auto
#plt.rcParams["animation.html"] = "jshtml"
FRAMES_PER_NUMBER = 20
MAX_NUMBER_TO_CHECK = 5
plotCenter = (0,0)
artists_to_draw_manually = []


fig = plt.figure()
ax = fig.subplots()

xSpine = ax.spines['bottom']
xSpine.set_position('zero')
#xSpine.set_bounds(0, None)

class CircleCycleForOnePrime:
	RING_WIDTH = 2
	INDICATOR_WIDTH = RING_WIDTH * 2
	ANGLE_TO_INDICATE_ZERO = 270.0
	center = (0,0)

	currentIndicatorValue = 0
	zeroWedge = None
	nonZeroWedge = None
	indicatorWedge = None
	unitSlice = None
	axesForCycle = None

	def __init__(self, axes):
		self.axesForCycle = axes
		self.constructPrimeModCirclePatches()
	
	def constructPrimeModCirclePatches(self):
		print("contruct prime ")

		self.zeroWedge=patchArc(self.center, 2, 2, theta1=0, theta2=180, color='red', animated=False)
		self.axesForCycle.add_patch(self.zeroWedge)

		self.nonZeroWedge=patchArc(self.center, 2, 2, theta2=0, theta1=180, color='green', animated=False)
		self.axesForCycle.add_patch(self.nonZeroWedge)
		
		self.indicatorWedge = patchArc(self.center, 2, 2, theta1= 260, theta2= 280, color='blue', animated=True)
		self.axesForCycle.add_patch(self.indicatorWedge)
		print("done construct circle")

#The main drawing method
def tick(currentNumber, oneCycle):
	# reset the background back in the canvas state, screen unchanged
	print("just reset the background")
	#only checxk for a new cycle if this is an integer
	oneCycle.indicatorWedge.set_angle(currentNumber*5)
	oneCycle.axesForCycle.draw_artist(oneCycle.indicatorWedge)
    # flush any pending GUI events, re-painting the screen if needed
	fig.canvas.flush_events()
	
plt.show(block=False)
print("about to make circle")
twoCycle = CircleCycleForOnePrime(ax)
ax.autoscale_view()
fig.canvas.flush_events()
backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)
print("done 2 cycle")

# show the result to the screen, this pushes the updated RGBA buffer from the
# renderer to the GUI framework so you can see it
fig.canvas.blit(fig.bbox)

#driving update loop
numberOfFrames = FRAMES_PER_NUMBER * MAX_NUMBER_TO_CHECK
for animationFrameIndex in range(numberOfFrames):
	fig.canvas.restore_region(backGroundImage)
	tick(animationFrameIndex, twoCycle)
    # copy the image to the GUI state, but screen might not be changed yet
#	backGroundImage = fig.canvas.copy_from_bbox(fig.bbox)
	fig.canvas.blit(fig.bbox)
	# you can put a pause in if you want to slow things down
	plt.pause(.1)

In [1]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib auto
#plt.rcParams["animation.html"] = "html5"


x = np.linspace(0, 2 * np.pi, 100)

fig, ax = plt.subplots()

numberDisplay = fig.text(.5, .9,'TEST BLIT', fontsize='large', animated=True)


# animated=True tells matplotlib to only draw the artist when we
# explicitly request it
(ln,) = ax.plot(x, np.sin(x), animated=True)

# make sure the window is raised, but the script keeps going
plt.show(block=False)

# stop to admire our empty window axes and ensure it is rendered at
# least once.
#
# We need to fully draw the figure at its final size on the screen
# before we continue on so that :
#  a) we have the correctly sized and drawn background to grab
#  b) we have a cached renderer so that ``ax.draw_artist`` works
# so we spin the event loop to let the backend process any pending operations
plt.pause(0.1)

# get copy of entire figure (everything inside fig.bbox) sans animated artist
bg = fig.canvas.copy_from_bbox(fig.bbox)
# draw the animated artist, this uses a cached renderer
ax.draw_artist(ln)
# show the result to the screen, this pushes the updated RGBA buffer from the
# renderer to the GUI framework so you can see it
fig.canvas.blit(fig.bbox)
for j in range(1000):
	newTextPosition = (.5+(j/1000), .9)
	numberDisplay.set_position(newTextPosition)
	# reset the background back in the canvas state, screen unchanged
	fig.canvas.restore_region(bg)
#	print("just restore")
#	plt.pause(.5)
	# update the artist, neither the canvas state nor the screen have changed
	ln.set_ydata(np.sin(x + (j / 100) * np.pi))
	bg = fig.canvas.copy_from_bbox(fig.bbox)
	# re-render the artist, updating the canvas state, but not the screen
	ax.draw_artist(numberDisplay)
	ax.draw_artist(ln)
#	print("just drew artists")
#	plt.pause(5)
	# copy the image to the GUI state, but screen might not be changed yet
	fig.canvas.blit(fig.bbox)
#	print("just blit")
#	plt.pause(.5)
	# flush any pending GUI events, re-painting the screen if needed
	fig.canvas.flush_events()
#	print("just flush")
#	plt.pause(.2)
	# you can put a pause in if you want to slow things down
	#plt.pause(.1)

Using matplotlib backend: <object object at 0x7f9939fef490>
